In [1]:
import torch
import torch.nn as nn
from utils import train
from bin_packing_dataset import BinPackingDataset
from bin_packing_model import BinPackingLSTMModel
from torch.utils.data import random_split

In [2]:
# Fijamos la semilla para que los resultados sean reproducibles
SEED = 23

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# Algunas constantes

# definimos el dispositivo que vamos a usar
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS

NUM_WORKERS = 0 # max(os.cpu_count() - 1, 1)  # número de workers para cargar los datos


print(f"Device: {DEVICE}")
print(f"Num Workers: {NUM_WORKERS}")

Device: mps
Num Workers: 0


### Exploración del Dataset

In [4]:
#Creacion del dataset de entrenamiento, validacion y test

full_dataset = BinPackingDataset('data')
print('Full dataset size:', len(full_dataset))
container_tensor, boxes_tensor = full_dataset[0]
print('Container:', container_tensor)
print('Boxes:', boxes_tensor)

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [int(0.7*len(full_dataset)), int(0.20*len(full_dataset)), int(0.10*len(full_dataset))])
print('Train dataset size:', len(train_dataset))
print('Val dataset size:', len(val_dataset))
print('Test dataset size:', len(test_dataset))

Full dataset size: 81000
Container: tensor([14., 11.])
Boxes: tensor([[ 8.,  7.],
        [12.,  2.],
        [ 1.,  2.],
        [ 2.,  7.],
        [ 4.,  8.]])
Train dataset size: 56700
Val dataset size: 16200
Test dataset size: 8100


In [5]:
# Collate para manejar secuencias de diferentes longitudes
import torch.nn.utils.rnn as rnn_utils

def custom_collate_fn_with_padding(batch):
    """
    Collate function que mantiene la estructura de contenedor y agrega padding a las secuencias de cajas.
    
    Args:
        batch (list): Lista de tuplas (contenedor, cajas).
        
    Returns:
        tuple: (contenedores, cajas_padded, longitudes) donde:
            - contenedores: Tensor de tamaño (batch_size, 2).
            - cajas_padded: Tensor de tamaño (batch_size, max_len, 2) con padding.
            - longitudes: Tensor de tamaños originales de las secuencias de cajas.
    """
    containers = torch.stack([item[0] for item in batch])  # Contenedores como tensor
    boxes = [item[1] for item in batch]  # Lista de cajas
    
    # Padding de las secuencias de cajas (rellenar con ceros hasta la longitud máxima en el batch)
    boxes_padded = rnn_utils.pad_sequence(boxes, batch_first=True)
    
    # Longitudes originales de cada secuencia de cajas
    lengths = torch.tensor([len(b) for b in boxes])
    
    return containers, boxes_padded



BATCH_SIZE = 500
mock_loader = torch.utils.data.DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn_with_padding)

x, y = next(iter(mock_loader))
print('Tamaño del primer contenedor:', x[0])
print('Tamaño de las cajas del primer contenedor:', y[0])


train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn_with_padding)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn_with_padding)

Tamaño del primer contenedor: tensor([14., 11.])
Tamaño de las cajas del primer contenedor: tensor([[ 8.,  7.],
        [12.,  2.],
        [ 1.,  2.],
        [ 2.,  7.],
        [ 4.,  8.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]])


### Primer modelo: Seq2Seq

La idea inicial es usar un Seq2Seq que pueda generar secuencias de cajas a partir del tamaño del contenedor

In [6]:
from bin_packing_seq_2_seq_solver import BinPackingSeq2SeqSolver
from utils import print_log

seq2seq = BinPackingSeq2SeqSolver(train_loader=train_dataloader, val_loader=val_dataloader, log_fn=print_log, device=DEVICE)

model, epoch_train_errors, epoch_val_errors = seq2seq.train()


/opt/anaconda3/envs/pytorch_m1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch: 002 | Train Loss: 5.28223 | Val Loss: 5.22082
Epoch: 004 | Train Loss: 5.21119 | Val Loss: 5.18844
Epoch: 006 | Train Loss: 5.21283 | Val Loss: 5.18860
Epoch: 008 | Train Loss: 5.17334 | Val Loss: 5.16877
Epoch: 010 | Train Loss: 5.18539 | Val Loss: 5.17353
Epoch: 012 | Train Loss: 5.17877 | Val Loss: 5.16997
Epoch: 014 | Train Loss: 5.18614 | Val Loss: 5.16890
Epoch: 016 | Train Loss: 5.17091 | Val Loss: 5.16022
Epoch: 018 | Train Loss: 5.19487 | Val Loss: 5.19058
Epoch: 020 | Train Loss: 5.17606 | Val Loss: 5.16174
Epoch: 022 | Train Loss: 5.16321 | Val Loss: 5.19332
Epoch: 024 | Train Loss: 5.17122 | Val Loss: 5.16046
Epoch: 026 | Train Loss: 5.17440 | Val Loss: 5.19140
Epoch: 028 | Train Loss: 5.16787 | Val Loss: 5.16237
Epoch: 030 | Train Loss: 5.16811 | Val Loss: 5.15758
Epoch: 032 | Train Loss: 5.17813 | Val Loss: 5.16229
Epoch: 034 | Train Loss: 5.17654 | Val Loss: 5.17877
Epoch: 036 | Train Loss: 5.19115 | Val Loss: 5.19917
Epoch: 038 | Train Loss: 5.18211 | Val Loss: 5

In [9]:
from models import BinPackingGame, Box, ResolvedBinPackingGameResult
dataset_keys = set()

def tensor_to_box(tensor):
    return Box(int(tensor[0].item()), int(tensor[1].item()))
    

for container_tensor, boxes_tensor in test_dataset:

    boxes = [tensor_to_box(tensor) for tensor in boxes_tensor]

    game = BinPackingGame(tensor_to_box(container_tensor), boxes)
    game_key = game.generate_unique_key()
    dataset_keys.add(game_key)

In [12]:
from typing import Counter
from models import BinPackingGame, Box, ResolvedBinPackingGameResult


model.eval()

container_width = 7
container_height = 5

attempts = 100
valid_games = 0
unique_games = set()
coverages = set()
boxes_count = Counter()
new_games = 0
for i in range(attempts):
    input = torch.tensor([[container_width, container_height]],dtype=torch.float32).to(DEVICE)
    output = model(input, 100)

    filtered_tensor = output[output != 0].view(-1, 2)

    # Paso 2: Convertir a lista de tuplas
    box_list = [tensor_to_box(tensor) for tensor in filtered_tensor]

    # boxes = [Box(int(box_width), int(box_height)) for(box_width, box_height) in box_list]

    valid_boxes = [box for box in box_list if box.width > 0 and box.height > 0]

    game = BinPackingGame(Box(container_width, container_height), valid_boxes)
    result = game.solve()
    if isinstance(result, ResolvedBinPackingGameResult):
        valid_games += 1
        game_key = game.generate_unique_key()
        boxes_count[len(game.boxes)] += 1
        # print(f'{game_key=}')
        if game_key not in unique_games:
            unique_games.add(game_key)
            coverages.add(game.coverage())
            if game_key not in dataset_keys:
                new_games += 1

print(f"Valid games: {valid_games}/{attempts}")
print(f"Unique games: {len(unique_games)}/{attempts}")
print(f"unique_games keys: {unique_games}")
print(f"Coverages: {coverages}")
print(f"New games: {new_games}")
print(f"Boxes count: {boxes_count}")
    




Valid games: 100/100
Unique games: 1/100
unique_games keys: {'4895df429991ff1f9e6f4bb161b49298'}
Coverages: {0.7428571428571429}
New games: 1
Boxes count: Counter({5: 100})
